In [12]:
import numpy as np
from hmmlearn import hmm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(42)


In [13]:
#read chord
chord = pd.read_csv('transition__chord_matrix/csv_file/all_chord.csv')
chord_list = chord['chord'].unique()
chord_list.sort()
chord_list = list(chord_list[:len(chord_list)-2])
        
pitch_names = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

In [14]:
#read transition matrix
transition_matrix = pd.read_csv('transition__chord_matrix/csv_file/transition_chord.csv')

#convert transition matrix to numpy array
transition_matrix.drop('Unnamed: 0', axis=1, inplace=True)
transition_matrix = transition_matrix.replace('%','',regex=True).astype('float')
transition_matrix = transition_matrix.div(transition_matrix.sum(axis=1), axis=0)



transition_matrix = transition_matrix.to_numpy()


transition_matrix

array([[0.017991  , 0.        , 0.07186407, ..., 0.        , 0.        ,
        0.005997  ],
       [0.        , 0.1282513 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00430172, 0.        , 0.10734294, ..., 0.        , 0.00190076,
        0.        ],
       ...,
       [0.        , 0.        , 0.01220732, ..., 0.06603962, 0.02931759,
        0.03422053],
       [0.02940588, 0.        , 0.01830366, ..., 0.02570514, 0.06421284,
        0.0495099 ],
       [0.        , 0.        , 0.        , ..., 0.02589223, 0.05618315,
        0.07337799]])

In [15]:
#create emission matrix

emission_matrix = pd.read_csv('transition_melody_matrix/csv_file/all_pitch_sorted.csv')

#convert  to numpy array
emission_matrix = emission_matrix.set_index('Unnamed: 0')

emission_matrix = emission_matrix.div(emission_matrix.sum(axis=1), axis=0)
emission_matrix = emission_matrix.to_numpy()

emission_matrix

array([[0.01289742, 0.0644871 , 0.32253549, ..., 0.13547291, 0.        ,
        0.04519096],
       [0.23084617, 0.02560512, 0.15383077, ..., 0.        , 0.02560512,
        0.02560512],
       [0.0101    , 0.086     , 0.1138    , ..., 0.127     , 0.004     ,
        0.1741    ],
       ...,
       [0.1554    , 0.        , 0.1779    , ..., 0.0576    , 0.1905    ,
        0.        ],
       [0.09349065, 0.00569943, 0.17278272, ..., 0.16428357, 0.0509949 ,
        0.14448555],
       [0.24487346, 0.01030309, 0.21036311, ..., 0.07242173, 0.01720516,
        0.0310093 ]])

In [16]:
#define the state
states = chord_list
n_states = len(states)

#define observation
observations_variable = pitch_names
n_observations = len(observations_variable)

#declare start probability
#each state has the same probability and sum to 1
start_probability = np.full(n_states, 1/n_states)

transition_probability = transition_matrix

emission_probability = emission_matrix



In [17]:

def midi_note_to_pitch(midi_note) -> str:
    """
    Convert midi note to pitch
    param midi_note: int
    return: str
    """

    # Equal temperament
    pitch_names = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    octave = (midi_note - 12) // 12 + 1
    pitch_class = midi_note % 12
    pitch_name = pitch_names[pitch_class]
    return f'{pitch_name}'
#read midi file
import pretty_midi

mid= pretty_midi.PrettyMIDI('midi_output.mid')
mid.instruments[0].notes
list_note = []

for i in mid.instruments[0].notes:
    # print(i.start, i.end , midi_note_to_pitch(i.pitch) )
    note = midi_note_to_pitch(i.pitch)
    list_note.append(pitch_names.index(note))


#create 2d np array consit of set of three notes
list_note = np.array(list_note)

#if the length of list_note is not divisible by 3, remove the last element
if len(list_note)%4 != 0: 
    list_note = list_note[:len(list_note)-1]

list_note = list_note.reshape(-1,4)




In [18]:
#assemble mutinomialHMM model
model = hmm.CategoricalHMM(n_components=n_states,n_iter=1000 )
model.startprob_ = start_probability
model.transmat_ = transition_probability
model.emissionprob_ = emission_probability

In [19]:
from music21 import converter, tempo

midi_file = 'midi_output.mid'
score = converter.parse(midi_file)

bpm = score.flat.getElementsByClass(tempo.MetronomeMark)[0].number
measure_list = []
measure_time_list = []
part = score.parts[0]
for measure in part.getElementsByClass("Measure"):
    
    list_note = []
    measure_time_list.append(measure.seconds)
    for note in measure.notes:
        note_name = note.pitch.midi
        list_note.append(pitch_names.index(midi_note_to_pitch(note_name)))
    
    #convet to numpy array
    list_note = np.array(list_note)
    measure_list.append(list_note)    
#convert to numpy array
measure_list = np.array(measure_list)
observations_sequence = measure_list

score.show('text')
measure_list

{0.0} <music21.metadata.Metadata object at 0x2164dc629a0>
{0.0} <music21.stream.Part 0x2164dc62370>
    {0.0} <music21.stream.Measure 1 offset=0.0>
        {0.0} <music21.instrument.ElectricPiano 'Electric Piano'>
        {0.0} <music21.clef.BassClef>
        {0.0} <music21.tempo.MetronomeMark animato Quarter=120.0>
        {0.0} <music21.meter.TimeSignature 4/4>
        {0.0} <music21.note.Rest 1.25ql>
        {1.25} <music21.note.Note C>
        {2.0} <music21.note.Note C>
        {2.3333} <music21.note.Note C>
        {2.5833} <music21.note.Rest 1/6ql>
        {2.75} <music21.note.Note C>
        {3.5} <music21.note.Note C>
    {4.0} <music21.stream.Measure 2 offset=4.0>
        {0.0} <music21.note.Note C>
        {0.5} <music21.note.Note A>
        {0.75} <music21.note.Rest 16th>
        {1.0} <music21.note.Note A>
        {1.75} <music21.note.Rest 1.25ql>
        {3.0} <music21.note.Note G>
        {3.3333} <music21.note.Note G>
        {3.75} <music21.note.Note G>
    {8.0} <musi

C:\Users\Hsieh\AppData\Local\Temp\ipykernel_21864\3578208893.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  measure_list = np.array(measure_list)


array([array([0, 0, 0, 0, 0]), array([0, 9, 9, 7, 7, 7]),
       array([7, 6, 0, 9, 9]), array([10,  0,  0,  0,  0,  0]),
       array([0, 0, 0, 9, 7]), array([7, 9, 3, 1, 2, 0, 0, 0])],
      dtype=object)

In [22]:
note_list = []
for i in range(len(measure_list)):
   
    note_list.append(np.bincount(measure_list[i]).argmax())
seen = np.array([note_list]).T
song_chord_list= model.predict(seen)


[ 94 100  48  93  29  75]


In [21]:

from pychord import Chord

chord_list = []
def convert_to_note_name(chord_str) -> str:
    """
    :param chord_name: str
    :return: str
    """
    chord_parts = chord_str.split(':')
    chord_name = chord_parts[0]  
    chord_type = chord_parts[1] 

    if 'min' in chord_type:
        #replace min with m
        chord_type = chord_type.replace('min', 'm')

    #if last character is 6
    if chord_type[-1] == '6':
        chord_type = chord_type.replace('6', '')
    #if last character is not num
    if chord_type[-1].isdigit() == False:
        if "maj" in chord_type:
            chord_type = chord_type.replace('maj', '')
    return chord_name + chord_type


for i in song_chord_list:
   
    c = Chord(convert_to_note_name(states[i]))
    chord_list.append( c.components())
chord_list


AttributeError: 'numpy.int64' object has no attribute 'split'

In [ ]:
from music21 import stream, chord, instrument, tempo, duration

def chords_to_midi(chords, file_name):
    # Create a stream object
    stream1 = stream.Stream()

    # Create an instrument
    piano = instrument.Piano()

    # Add the instrument to the stream
    stream1.append(piano)

    # Set the duration for each chord
    duration_obj = duration.Duration('whole')

    # Create chord objects and add them to the stream
    for chord_notes in chords:
        chord_obj = chord.Chord(chord_notes, duration=duration_obj)
        stream1.append(chord_obj)

    # Write the stream to a MIDI file
    midi_file_path = file_name + '.mid'
    stream1.write('midi', fp=midi_file_path)

# Test the code
chords = chord_list
file_name = 'chords_output'

chords_to_midi(chords, file_name)
